# General Imports

**Importing all libraries**

In [34]:
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
import warnings

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

import plotly.graph_objects as go
from bayes_opt import BayesianOptimization
import matplotlib.pyplot as plt


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot

import statsmodels.api as sm

import plotly.express as px
import plotly.graph_objects as go

from scipy import stats
from scipy.stats import mode

from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, StratifiedKFold
from sklearn import linear_model
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, recall_score, precision_score, f1_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.inspection import permutation_importance

from bayes_opt import BayesianOptimization


In [ ]:
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
import warnings

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

import plotly.graph_objects as go
from bayes_opt import BayesianOptimization
import matplotlib.pyplot as plt

import seaborn as sns
from matplotlib import pyplot
import statsmodels.api as sm
import plotly.express as px
from scipy import stats
from scipy.stats import mode
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn import metrics
from sklearn.inspection import permutation_importance


Disable warnings

In [35]:
# Disable all warnings
warnings.filterwarnings("ignore")

# Enable warnings again
## warnings.filterwarnings("default")

Generate train and test data

In [36]:
# Read Data
df = pd.read_csv("data_finish_prep.csv")

# Select only the usefull part of the df
df.drop(columns=['verschil_Lengte',
               'verschil_6 MWT', 'verschil_TUG',
               'verschil_BMI', 'verschil_Conditie',
               'verschil_Lenigheid', 'verschil_Knijpkracht'], inplace=True)

# Drop NaN values
df.dropna(inplace=True)

# Define X and y
X = df.drop(columns=['verschil_Gewicht'])
y = df['verschil_Gewicht']

# Split data into train & test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Assuming you have X_train, y_train defined for regression
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return -scores.mean()

# MLR

**feature selection**

In [38]:
# Fit the Full Model
X =  sm.add_constant(X_train)  # Add constant term for the intercept
model = sm.OLS(y_train, X_train).fit()  # Fit the MLR model
best_model = model  # Initialize the best model
selected_features = X_train.columns.tolist()
best_features = X_train.columns.tolist()
best_features_rmse = np.sqrt(-cross_val_score(LinearRegression(), X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean())

# Iteratively remove one feature at a time based on p-values
while True:
    # Compute p-values
    p_values = model.pvalues[1:]  # Exclude the constant term

    # Identify Insignificant Features
    insignificant_feature = p_values.idxmax()
    max_p_value = p_values.max()

    # Remove Insignificant Feature
    X = X.drop(insignificant_feature, axis=1)
    selected_features.remove(insignificant_feature)

    # When there a no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    model = sm.OLS(y_train, X).fit()
    
    # Evaluate Model Performance
    rmse = np.sqrt(-cross_val_score(LinearRegression(), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error').mean())

    # Check if the model performs better without the insignificant features
    if rmse <= best_features_rmse:
        best_features_rmse = rmse
        best_features = selected_features.copy()
        best_model = model

# Save results
best_model_mlr = LinearRegression().fit(X_train[best_features], y_train)
best_features_mlr = best_features

# Show results
print("Best features:", best_features)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features)))

Best features: ['AfstandTotRestaurant', 'AfstandTotSportterrein', 'AfstandTotKunstijsbaan', 'AfstandTotMuseum', 'AfstandTotPodiumkunstenTotaal', 'AfstandTotBioscoop', 'AfstandTotSauna', 'BeperkingInZien', 'VoldoetAanBeweegrichtlijn', 'UrenMantelzorgPerWeek']
Dropped features: ['AfstandTotAttractie', 'EenOfMeerLangdurigeAandoeningen', 'kPersonenMetLaagsteInkomen', 'AfstandTotBuitenschoolseOpvang', 'BouwjaarVanaf2000', 'AfstandTotPoppodium', 'AfstandTotCafetariaED', 'AfstandTotDagrecreatiefTerrein', 'EigendomOnbekend', 'PersonenautoSPerHuishouden', 'AfstandTotOpenNatNatuurlijkTerrein', 'AfstandTotVolkstuin', 'AfstandTotOpenDroogNatTerrein', 'AfstandTotConsultatiebureau', 'BeperkingInHoren', 'OvermatigDrinker', 'ErvarenGezondheidGoedZeerGoed', 'AfstandTotBibliotheek', 'AfstandTotOpenbaarGroenTotaal', 'GeboorteRelatief', 'AfstandTotHuisartsenpraktijk', 'AfstandTotZwembad', 'AfstandTotBos', 'InBezitOverigeVerhuurders', 'GeweldsEnSeksueleMisdrijven', 'PercentageOnbewoond', 'AfstandTotHotelED

*Er zijn geen hyperparameters om te optimaliseren.*

# Support Vector Machines

**Hyperparameter optimalisatie**

In [39]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    classifier = model(**params)
    
    # Calculating accuracy based on cross-validation
    scores = cross_val_score(classifier, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
    return scores.mean()

params_ranges = {
    'C': (0.1, 10),
    'kernel_int': (1, 4),
    'gamma': (0.001, 0.1)
}

kernel_mapping 'linear',
    2: 'rbf',
    3: 'poly',
    4: 'sigmoid'= {
    1: 
}

# Example usage with SVM
model = SVR
model_bo = BayesianOptimization(f=lambda C, kernel_int, gamma:
                                    bo_params_generic(model, {
                                        'C': C,
                                        'kernel': kernel_mapping[int(kernel_int)],
                                        'gamma': gamma
                                    }, X_train, y_train),
                             pbounds=params_ranges)

results = model_bo.maximize(n_iter=40, init_points=10)
params = model_bo.max['params']

# Creating a model with the best hyperparameters
best_model_svm = model(
    C=params['C'],
    kernel=kernel_mapping[int(params['kernel_int'])],
    gamma=params['gamma']
)

# Fit the model
best_model_svm.fit(X_train, y_train)


|   iter    |  target   |     C     |   gamma   | kernel... |
-------------------------------------------------------------
| 1         | -11.63    | 9.469     | 0.04878   | 3.63      |
| 2         | -11.63    | 5.625     | 0.09757   | 2.575     |
| 3         | -11.63    | 0.5236    | 0.03564   | 1.344     |
| 4         | -11.66    | 4.537     | 0.05762   | 1.631     |
| 5         | -11.66    | 7.067     | 0.03837   | 1.562     |
| 6         | -11.64    | 5.473     | 0.06367   | 3.283     |
| 7         | -11.62    | 2.223     | 0.02886   | 2.415     |
| 8         | -11.63    | 3.309     | 0.03722   | 3.316     |
| 9         | -11.63    | 7.955     | 0.07763   | 2.329     |
| 10        | -11.65    | 1.978     | 0.06698   | 1.81      |
| 11        | -11.62    | 7.376     | 0.01786   | 2.386     |
| 12        | -11.66    | 9.78      | 0.04432   | 1.238     |
| 13        | -11.62    | 2.43      | 0.04866   | 2.487     |
| 14        | -11.62    | 1.985     | 0.08665   | 2.489     |
| 15    

SVR(C=9.873984723455573, gamma=0.009086011668760496, kernel='poly')

**Feature Selection**

In [40]:
# Get the accuracy score of the model with all features
best_accuracy = cross_val_score(best_model_svm, X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()
best_model = best_model_svm
selected_features = X_train.columns
best_features = selected_features

while len(selected_features) > 1:
    try:
        # Retrieve the coefficients from the SVM model
        coefficients = best_model_svm.coef_[0]

        # Sort features based on their coefficient magnitudes
        sorted_indices = np.argsort(np.abs(coefficients))
        selected_features = [X_train.columns[i] for i in sorted_indices[::-1]]

        # Remove the least important feature
        selected_features = selected_features[:-1]
        
        # Refit the linear SVM classifier
        best_model_svm_classification.fit(X_train[selected_features], y_train)
        
        # Evaluate Model Performance
        accuracy = cross_val_score(best_model_svm, X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

        # Check if the model performs better without the insignificant features
        if accuracy >= best_accuracy:
            best_accuracy = accuracy
            best_features = selected_features.copy()
            best_model = best_model_svm
    except:
        print("Scince the kernal is not linear, this feature selection is not possible") 
        break

# Save results
best_model_svm = best_model
best_features_svm = best_features

# Show results
print("Best features:", best_features_svm)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_svm)))

Scince the kernal is not linear, this feature selection is not possible
Best features: Index(['NederlandseAntillenEnAruba', 'GeboorteRelatief', 'SterfteRelatief',
       'GemiddeldeWoningwaarde', 'PercentageOnbewoond',
       'InBezitWoningcorporatie', 'InBezitOverigeVerhuurders',
       'EigendomOnbekend', 'BouwjaarVanaf2000', 'kPersonenMetLaagsteInkomen',
       'ALandbouwBosbouwEnVisserij', 'KLFinancieleDienstenOnroerendGoed',
       'PersonenautoSOverigeBrandstof', 'PersonenautoSPerHuishouden',
       'PersonenautoSNaarOppervlakte', 'AfstandTotHuisartsenpraktijk',
       'AfstandTotKinderdagverblijf', 'AfstandTotSchool', 'OppervlakteWater',
       'TotaalDiefstalUitWoningSchuurED',
       'VernielingMisdrijfTegenOpenbareOrde', 'GeweldsEnSeksueleMisdrijven',
       'AfstandTotHuisartsenpost', 'AfstandTotApotheek',
       'AfstandTotConsultatiebureau', 'AfstandTotOvDagelLevensmiddelen',
       'AfstandTotWarenhuis', 'AfstandTotCafeED', 'AfstandTotCafetariaED',
       'AfstandTotResta

# Random Forest

**Feature selection**

In [41]:
# Get the accuracy score of the model with all the features
rf = RandomForestRegressor(n_estimators=100, random_state=0).fit(X_train, y_train)
best_rf = rf
selected_features = X_train.columns.tolist()
best_features = selected_features
best_features_accuracy = cross_val_score(RandomForestRegressor(n_estimators=100, random_state=0), X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

while True:
    # Make DataFrame of feature impotances
    result = permutation_importance(
            rf, X_train[selected_features], y_train, n_repeats=10, random_state=42, n_jobs=-1
        )
    df_importances = pd.DataFrame({'feature': X_train[selected_features].columns, 'importance': result.importances_mean, 'std': result.importances_std})
    df_importances = df_importances.sort_values('importance')

    # Remove least important feature
    selected_features.remove(df_importances['feature'].iloc[0])

    # When there are no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    rf = RandomForestRegressor(n_estimators=100, random_state=0).fit(X_train[selected_features], y_train)

    # Evaluate Model Performance
    accuracy = cross_val_score(RandomForestRegressor(n_estimators=100, random_state=0), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

    # Check if the model performs better without the insignificant features
    if accuracy >= best_features_accuracy:
        best_features_accuracy = accuracy
        best_features = selected_features.copy()
        best_model = rf

# Save results
best_model_rf = best_model
best_features_rf = best_features

# Show results
print("Best features:", best_features_rf)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_rf)))

Best features: ['AfstandTotTreinstationsTotaal', 'AfstandTotZonnebank']
Dropped features: ['AfstandTotAttractie', 'EenOfMeerLangdurigeAandoeningen', 'kPersonenMetLaagsteInkomen', 'AfstandTotBuitenschoolseOpvang', 'AfstandTotSauna', 'AfstandTotMuseum', 'BouwjaarVanaf2000', 'AfstandTotPoppodium', 'AfstandTotCafetariaED', 'BeperkingInZien', 'AfstandTotDagrecreatiefTerrein', 'EigendomOnbekend', 'PersonenautoSPerHuishouden', 'AfstandTotOpenNatNatuurlijkTerrein', 'AfstandTotVolkstuin', 'AfstandTotOpenDroogNatTerrein', 'AfstandTotConsultatiebureau', 'BeperkingInHoren', 'AfstandTotRestaurant', 'UrenMantelzorgPerWeek', 'OvermatigDrinker', 'ErvarenGezondheidGoedZeerGoed', 'AfstandTotBibliotheek', 'AfstandTotOpenbaarGroenTotaal', 'AfstandTotPodiumkunstenTotaal', 'GeboorteRelatief', 'AfstandTotHuisartsenpraktijk', 'AfstandTotBioscoop', 'AfstandTotZwembad', 'AfstandTotBos', 'InBezitOverigeVerhuurders', 'GeweldsEnSeksueleMisdrijven', 'PercentageOnbewoond', 'AfstandTotHotelED', 'AfstandTotWarenhuis',

**Hyperparameter optimalisatie**

In [42]:
def bo_params_rf(min_samples_split,max_depth,max_features,n_estimators):
    
    params = {
        'min_samples_split': round(min_samples_split),
        'max_depth': round(max_depth),
        'max_features': max_features,
        'n_estimators':round(n_estimators)
    }    
        
    clf = RandomForestRegressor(min_samples_split=int(params['min_samples_split']), 
                                 max_depth=int(params['max_depth']), 
                                 max_features=params['max_features'],
                                 n_estimators=int(params['n_estimators']), 
                                 bootstrap=False,
                                 n_jobs = -1)
    return cross_val_score(clf, X_train[best_features_rf], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

# Create Bayesian Optimization
rf_bo = BayesianOptimization(f=bo_params_rf, pbounds={
    'n_estimators': (100, 1000),
    'max_depth': (1, 20),
    'max_features': (0.1, 1),
    'min_samples_split': (2, 12)
})

# Bayesian optimization
results = rf_bo.maximize(n_iter=40, init_points=10)

# Getting best hyperparameters from bayesian optimization
params = rf_bo.max['params']
params['n_estimators']= round(params['n_estimators'])
params['min_samples_split']= round(params['min_samples_split'])
params['max_depth']= round(params['max_depth'])

# Making random forest model with the best hyperparameters
# criterion entropy is after the hyperparameter optimization since it's more accurate but increases training time
best_model_rf_classification = RandomForestRegressor(min_samples_split=params['min_samples_split'],
                                                max_depth=params['max_depth'],
                                                max_features=params['max_features'],
                                                n_estimators=params['n_estimators'],
                                                bootstrap=False)

best_model_rf_classification.fit(X_train[best_features_rf], y_train)

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------


| 1         | -11.45    | 6.09      | 0.1975    | 10.19     | 889.2     |
| 2         | -11.49    | 3.423     | 0.1755    | 6.768     | 604.6     |
| 3         | -11.4     | 1.17      | 0.4598    | 3.312     | 535.7     |
| 4         | -11.44    | 17.01     | 0.8947    | 4.264     | 982.1     |
| 5         | -11.49    | 2.618     | 0.8813    | 8.614     | 994.3     |
| 6         | -11.44    | 8.151     | 0.397     | 10.54     | 985.5     |
| 7         | -11.44    | 16.27     | 0.1455    | 11.93     | 327.1     |
| 8         | -11.44    | 1.982     | 0.6501    | 3.436     | 180.9     |
| 9         | -11.44    | 2.457     | 0.5042    | 10.14     | 844.3     |
| 10        | -11.44    | 11.5      | 0.2246    | 10.82     | 284.5     |
| 11        | -11.44    | 18.85     | 0.6649    | 4.393     | 239.9     |
| 12        | -11.44    | 18.62     | 0.8436    | 6.019     | 852.3     |
| 13        | -11.44    | 9.524     | 0.2059    | 10.96     | 953.3     |
| 14        | -11.44    | 11.13     | 

RandomForestRegressor(bootstrap=False, max_depth=1,
                      max_features=0.5137722493710422, min_samples_split=6,
                      n_estimators=543)

# neural network

**Feature selection**

In [43]:
# Get the accuracy score of the model with all the features
mlp = MLPRegressor(random_state=0).fit(X_train, y_train)
best_model = mlp
selected_features = X_train.columns.tolist()
best_features = selected_features
best_features_accuracy = cross_val_score(MLPRegressor(random_state=0), X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

while True:
    # Retrieve the learned weights
    weights = mlp.coefs_[0]  # Weights connecting input features to the first hidden layer

    # Normalize the weights
    normalized_weights = np.abs(weights) / np.sum(np.abs(weights), axis=0)

    # Calculate feature importance
    feature_importance = np.mean(normalized_weights, axis=1)

    # Sort features based on their importance
    sorted_indices = np.argsort(feature_importance)
    selected_features = [selected_features[i] for i in sorted_indices[:-1]]

    # When there are no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    mlp = MLPRegressor(random_state=0).fit(X_train[selected_features], y_train)

    # Evaluate Model Performance
    accuracy = cross_val_score(MLPRegressor(random_state=0), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

    # Check if the model performs better without the insignificant features
    if accuracy >= best_features_accuracy:
        best_features_accuracy = accuracy
        best_features = selected_features.copy()
        best_model = mlp

# Save results
best_model_nn = best_model
best_features_nn = best_features

# Show results
print("Best features:", best_features_nn)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_nn)))

Best features: ['AfstandTotCafetariaED']
Dropped features: ['AfstandTotAttractie', 'EenOfMeerLangdurigeAandoeningen', 'kPersonenMetLaagsteInkomen', 'AfstandTotBuitenschoolseOpvang', 'AfstandTotSauna', 'AfstandTotMuseum', 'BouwjaarVanaf2000', 'AfstandTotPoppodium', 'BeperkingInZien', 'AfstandTotDagrecreatiefTerrein', 'EigendomOnbekend', 'PersonenautoSPerHuishouden', 'AfstandTotOpenNatNatuurlijkTerrein', 'AfstandTotVolkstuin', 'AfstandTotOpenDroogNatTerrein', 'AfstandTotConsultatiebureau', 'BeperkingInHoren', 'AfstandTotRestaurant', 'UrenMantelzorgPerWeek', 'OvermatigDrinker', 'ErvarenGezondheidGoedZeerGoed', 'AfstandTotBibliotheek', 'AfstandTotOpenbaarGroenTotaal', 'AfstandTotPodiumkunstenTotaal', 'GeboorteRelatief', 'AfstandTotHuisartsenpraktijk', 'AfstandTotBioscoop', 'AfstandTotZwembad', 'AfstandTotBos', 'InBezitOverigeVerhuurders', 'GeweldsEnSeksueleMisdrijven', 'PercentageOnbewoond', 'AfstandTotHotelED', 'AfstandTotWarenhuis', 'AfstandTotSchool', 'Ondergewicht', 'MatigOvergewicht',

**Hyperparameter optimalisatie**

In [44]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Calculating rmse based on
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return scores.mean()

params_ranges = {
    'hidden_layer_sizes': (10, 100),
    'alpha': (0.0001, 0.1),
    'learning_rate_init': (0.001, 0.1),
    'max_iter': (100, 1000)
}

# Example usage with Neural Network
model = MLPRegressor
model_bo = BayesianOptimization(f=lambda hidden_layer_sizes, alpha, learning_rate_init, max_iter:
                                    bo_params_generic(model, {
                                        'hidden_layer_sizes': (int(round(hidden_layer_sizes)),),
                                        'alpha': alpha,
                                        'learning_rate_init': learning_rate_init,
                                        'max_iter': int(round(max_iter))
                                    }, X_train[best_features_nn], y_train),
                             pbounds=params_ranges)

results = model_bo.maximize(n_iter=40, init_points=10)
params = model_bo.max['params']

# Creating a model with the best hyperparameters
best_model_nn = model(
    hidden_layer_sizes=(int(round(params['hidden_layer_sizes'])),),
    alpha=params['alpha'],
    learning_rate_init=params['learning_rate_init'],
    max_iter=int(round(params['max_iter']))
)

# Fit the model
best_model_nn.fit(X_train[best_features_nn], y_train)

|   iter    |  target   |   alpha   | hidden... | learni... | max_iter  |
-------------------------------------------------------------------------
| 1         | -3.347    | 0.08506   | 19.2      | 0.01035   | 573.3     |
| 2         | -3.349    | 0.04066   | 58.81     | 0.04671   | 986.4     |
| 3         | -3.358    | 0.08994   | 78.98     | 0.09923   | 664.3     |
| 4         | -3.357    | 0.0507    | 82.34     | 0.05477   | 215.9     |
| 5         | -3.35     | 0.0701    | 37.25     | 0.01183   | 734.9     |
| 6         | -3.355    | 0.02041   | 46.33     | 0.04747   | 945.9     |
| 7         | -3.353    | 0.07767   | 64.76     | 0.0491    | 738.9     |
| 8         | -3.362    | 0.02314   | 72.35     | 0.05508   | 523.8     |
| 9         | -3.353    | 0.08948   | 15.25     | 0.07658   | 623.0     |
| 10        | -3.353    | 0.04144   | 86.78     | 0.07472   | 331.9     |
| 11        | -3.35     | 0.06723   | 29.64     | 0.02028   | 587.1     |
| 12        | -3.365    | 0.04941   | 

MLPRegressor(alpha=0.028732258891203594, hidden_layer_sizes=(79,),
             learning_rate_init=0.05495494366203554, max_iter=714)

# plotting the bar chart

**using train dataset**

In [45]:
regression_models = [
    {
        'name': 'Multiple Linear Regression',
        'model': best_model_mlr,
        'features': best_features_mlr
    },
    {
        'name': 'Random Forest',
        'model': best_model_rf,
        'features': best_features_rf
    },
    {
        'name': 'Neural Network',
        'model': best_model_nn,
        'features': best_features_nn
    },
    {
        'name': 'Support Vector Machine',
        'model': best_model_svm,
        'features': best_features_svm
    }
]

regression_stats_val = []

for model in regression_models:
    try:
        # Perform cross-validation and calculate predictions and metrics
        y_pred = cross_val_predict(model['model'], X_train[model['features']], y_train, cv=10)
        y_true = y_train
    except: # cv can't be higher then n_splits in the dt
        # Perform cross-validation and calculate predictions and metrics
        y_pred = cross_val_predict(model['model'], X_train[model['features']], y_train, cv=5)
        y_true = y_train

    # R2 Score
    r2 = r2_score(y_true, y_pred)

    # Adjusted R2 Score
    n = len(y_true)  # Number of samples
    p = X_test.shape[1]  # Number of predictors (features) in X
    adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

    # Save stats
    regression_stats_val.append(
        {
            'model': model['name'],
            'mae': mean_absolute_error(y_true, y_pred),
            'mse': mean_squared_error(y_true, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            'r2': r2,
            'adj_r2': adj_r2
        }
    )

# View stats
regression_stats_val

[{'model': 'Multiple Linear Regression',
  'mae': 2.4324810322722312,
  'mse': 11.407483187566488,
  'rmse': 3.3774965858704413,
  'r2': 0.001819433776709789,
  'adj_r2': -0.07363698051662748},
 {'model': 'Random Forest',
  'mae': 2.444316719603332,
  'mse': 11.431662965555118,
  'rmse': 3.381074232482203,
  'r2': -0.0002963514571576731,
  'adj_r2': -0.07591270631892977},
 {'model': 'Neural Network',
  'mae': 2.4328018364948565,
  'mse': 11.472097018233946,
  'rmse': 3.387048422776673,
  'r2': -0.003834422470196275,
  'adj_r2': -0.07971823410401235},
 {'model': 'Support Vector Machine',
  'mae': 2.387038367026042,
  'mse': 11.592430604582425,
  'rmse': 3.4047658663383045,
  'r2': -0.014363883297096702,
  'adj_r2': -0.09104365849234153}]

In [46]:
# Extracting model names and metric scores
model_names = [d['model'] for d in regression_stats_val]
metric_labels = list(regression_stats_val[0].keys())[1:]
metric_scores = {metric: [d[metric] for d in regression_stats_val] for metric in metric_labels}

# Plotting the bar chart
fig = go.Figure()

for metric in metric_labels:
    fig.add_trace(go.Bar(
        x=model_names,
        y=metric_scores[metric],
        name=metric
    ))

# Updating the layout
fig.update_layout(
    title='Evaluation Metrics Comparison',
    xaxis_title='Models',
    yaxis_title='Scores',
    barmode='group'
)

# Display the plot
fig.show()

**Evaluating on test data**

In [48]:
regression_models = [
    {
        'name': 'Multiple Linear Regression',
        'model': best_model_mlr,
        'features': best_features_mlr
    },
    {
        'name': 'Random Forest',
        'model': best_model_rf,
        'features': best_features_rf
    },
    {
        'name': 'Neural Network',
        'model': best_model_nn,
        'features': best_features_nn
    },
    {
        'name': 'Support Vector Machine',
        'model': best_model_svm,
        'features': best_features_svm
    }
]

regression_stats_test = []

for model in regression_models:
    # Make predictions on the test set
    y_pred = model['model'].predict(X_test[model['features']])

    # Defining Actual values
    y_true = y_test.copy()

    # R2 Score
    r2 = r2_score(y_true, y_pred)

    # Adjusted R2 Score
    n = len(y_true)  # Number of samples
    p = X_test.shape[1]  # Number of predictors (features) in X
    adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

    # Save stats
    regression_stats_test.append(
        {
            'model': model['name'],
            'mae': mean_absolute_error(y_true, y_pred),
            'mse': mean_squared_error(y_true, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            'r2': r2,
            'adj_r2': adj_r2
        }
    )

# View stats
regression_stats_test

[{'model': 'Multiple Linear Regression',
  'mae': 2.7085294488746827,
  'mse': 15.159529909991816,
  'rmse': 3.8935240990639595,
  'r2': -0.006618760105282728,
  'adj_r2': -0.40026855455986254},
 {'model': 'Random Forest',
  'mae': 2.6953939551701644,
  'mse': 15.035564193768085,
  'rmse': 3.8775719456598203,
  'r2': 0.0016127758527318958,
  'adj_r2': -0.3888179821936859},
 {'model': 'Neural Network',
  'mae': 2.715724947759168,
  'mse': 15.075797514041405,
  'rmse': 3.8827564324898627,
  'r2': -0.001058785548506247,
  'adj_r2': -0.39253428827697245},
 {'model': 'Support Vector Machine',
  'mae': 2.699337207626536,
  'mse': 15.845489347425053,
  'rmse': 3.9806393139073846,
  'r2': -0.05216764206213331,
  'adj_r2': -0.4636298484551462}]

In [49]:
# Extracting model names and metric scores
model_names = [d['model'] for d in regression_stats_test]
metric_labels = list(regression_stats_test[0].keys())[1:]
metric_scores = {metric: [d[metric] for d in regression_stats_test] for metric in metric_labels}

# Plotting the bar chart
fig = go.Figure()

for metric in metric_labels:
    fig.add_trace(go.Bar(
        x=model_names,
        y=metric_scores[metric],
        name=metric
    ))

# Updating the layout
fig.update_layout(
    title='Evaluation Metrics Comparison',
    xaxis_title='Models',
    yaxis_title='Scores',
    barmode='group'
)

# Display the plot
fig.show()

Naive bayes

In [51]:
# Defining Actual values
y_true = y_test.copy()

# Defining Predicted values
y_pred = np.full_like(y_test, y_train.mean())

# Mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

# Mean Squared Error
mse = mean_squared_error(y_true, y_pred)
print("Mean Squared Error:", mse)

# Root Mean Squared Error
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

# R2 Score
r2 = r2_score(y_true, y_pred)
print("R2 Score:", r2)

# Adjusted R2 Score
n = len(y_true)  # Number of samples
p = X_test.shape[1]  # Number of predictors (features) in X
adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)
print("Adjusted R2 Score:", adj_r2)

Mean Absolute Error: 2.717184553660983
Mean Squared Error: 15.253915202666954
Root Mean Squared Error: 3.905626096116595
R2 Score: -0.012886105257073899
Adjusted R2 Score: -0.4089868168101196


Save best model as Pickle